<a href="https://www.kaggle.com/code/edmilsoneddi/discipline-of-artificial-intelligence-second-work?scriptVersionId=93669673" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Faculdade Metodista Granbery – Sistemas de Informação
## Inteligência Artifical –  [Prof. Ricardo Campos](https://www.linkedin.com/in/ricardo-campos-505220237/)
### Segundo trabalho 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

# for interactive visualizations
import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly import tools
init_notebook_mode(connected = True)
import plotly.figure_factory as ff

In [ ]:
#Adicionar o arquivo
data_netflix = pd.read_csv('../input/netflix-shows/netflix_titles.csv')
data_netflix[:5]

In [ ]:
data_netflix.info()

In [ ]:
data_netflix.head()

In [ ]:
data_netflix.isnull().sum()

In [ ]:
data_netflix.fillna(value='Ausente')

Filmes e Séries

In [ ]:
labels = ['Filmes', 'Séries']
size = data_netflix['type'].value_counts()
colors = ['lightgreen', 'orange']
explode = [0, 0.1]

plt.rcParams['figure.figsize'] = (10, 10)
plt.pie(size, colors = colors, explode = explode, labels = labels, shadow = True, autopct = '%.2f%%')
plt.title('Filmes e Séries', fontsize = 34)
plt.axis('off')
plt.legend()
plt.show()

Trocar nomes das colunas

In [ ]:
data_netflix = data_netflix.rename(columns={'date_added': 'add', 'duration': 'duration_seconds', 'listed_in': 'gender_type', 'type': 'movie_or_serie'})
data_netflix[:5]

In [ ]:
data_netflix['gender_type']

In [ ]:
df_split = data_netflix['gender_type'].str.split(',', expand=True)

In [ ]:
df_split

In [ ]:
df_split= df_split.fillna('VAZIO')

In [ ]:
t1 = lambda x: x.strip()
df_split[0] = df_split[0].apply(t1)
df_split[1] = df_split[1].apply(t1)
df_split[2] = df_split[2].apply(t1)

In [ ]:
df0 = pd.get_dummies(df_split[0]) 
df0

In [ ]:
df1 = pd.get_dummies(df_split[1]) 
df1

In [ ]:
df2 = pd.get_dummies(df_split[2]) 
df2

In [ ]:
#esta função faz merge entre dois datasets - se uma coluna existe nos dois, elas são somadas
def my_merge(dfA, dfB):
    #cria e inicializa o dataframe com a cópia do segundo 
    df_resultado = dfB.copy()
    #percorre a lista de colunas do primeiro dataset
    for colA in dfA.columns:
        eh_igual  = False
        #percorre a lista de colunas do segundo dataset
        for colB in dfB.columns:
            if(colA==colB):
                #se a coluna é igual, altera a flag pra true
                eh_igual = True
        #se a coluna existe nos dois dataset de entrada, soma
        if(eh_igual):
            df_resultado[colA] = dfA[colA]+dfB[colA]
        else:
            #se a coluna do primeiro dataset não existe no segundo, adiciona na saída
            df_resultado[colA] = dfA[colA]
    return df_resultado

In [ ]:
df01 = my_merge(df0, df1)
df01

In [ ]:
df012= my_merge(df2, df01)
df012

In [ ]:
#acrescestei o título e a coluna origina de genero no dataset com as dummies - pois depois vai facilitar a visualização.
# porém, na hora de fitar  o kmeans, passa um cópia do dataframe com estas colunas dropadas: 
#df012.drop(['title', 'gender_type'],axis=1)
df012['title'] = data_netflix['title'].str.upper()
df012['gender_type'] = data_netflix['gender_type']
df012

In [ ]:
#da pra ver quantos filmes tinham nenhuma, uma ou duas categorias em branco:
df012['VAZIO'].value_counts()

In [ ]:
#dropei a coliuna VAZIO pq ela não tem significado nenhum para o filme.
df012 = df012.drop('VAZIO', axis=1)
#df012_dropado = df012.drop(['title', 'gender_type'],axis=1)
#df012_dropado[115:124]

In [ ]:
df_7 = df012.copy()
df_8 = df012.copy()

KMEANS

In [ ]:
X = np.array(df012.drop(['title', 'gender_type'], axis=1))
X

Elbow - Cotovelo

In [ ]:
from sklearn.cluster import KMeans
array_ale = []
 
for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, init = 'random')
    kmeans.fit(X)
    array_ale.append(kmeans.inertia_)  


In [ ]:
plt.plot(range(1, 11), array_ale)
plt.title('O Metodo Elbow')
plt.xlabel('Numero de Clusters')
plt.ylabel('array_ale') #within cluster sum of squares
plt.show()

Escolhendo a melhor Cluster

Utilizando 7 Clusters

In [ ]:
kmeans_model = KMeans(n_clusters = 7, random_state=0)
y_Kmeans = kmeans_model.fit_predict(X)
y_Kmeans

In [ ]:
df_7['K-Classes'] = y_Kmeans

In [ ]:
df_7[:3]

Verificando a saída do K-Classes

In [ ]:
df_7[df_7['title'] ==  'NARCOS' ][['K-Classes', 'gender_type']]

In [ ]:
df_7[df_7['K-Classes'] ==  1 ][['title', 'gender_type']]

Utilizando 8 Clusters

In [ ]:
kmeans_model = KMeans(n_clusters = 8, random_state=0)
g_Kmeans = kmeans_model.fit_predict(X)
g_Kmeans

In [ ]:
df_8['K-Classes'] = g_Kmeans

In [ ]:
df_8[:3]

Verificando a saída do K-Classes

In [ ]:
df_8[df_8['title'] ==  'NARCOS' ][['K-Classes', 'gender_type']]

Utilzando o 8 Clusters o Kmeans obtve maior precisão, em seus dados,
trazendo resultados bem mais próximos que o dataset(df_7)
utilizando apenas 7 Clusters

In [ ]:
df_8[df_8['K-Classes'] ==  5 ][['title', 'gender_type']][:20]

Sistema de recomendação de Filmes e Séries

In [ ]:
def recomenda_filme(nome):
    nome_pesquisa = nome.upper()
    # Recebe o valor da id K-Classes
    filme = df_8[df_8['title'] == nome_pesquisa][['K-Classes']]
    # resetar o index, como possui apenas um linha, acessar mais rápido
    reseta_filme = filme.reset_index()
    # Adicionar a célula da K-Classes, a uma variável
    resetado_filme = reseta_filme.at[0, 'K-Classes']
    # Transforma variável string em inteiro
    k_id = int(resetado_filme)
    # Procura filmes com semelhança usando a k-id = K-Classes
    recomendacoes = df_8[df_8['K-Classes'] == k_id][['title', 'gender_type']][:10]
    return recomendacoes
recomenda_filme('Narcos')